# Interactive Neuroscope

*This is an interactive accompaniment to [neuroscope.io](https://neuroscope.io) and to the [studying learned language features post](https://www.alignmentforum.org/posts/Qup9gorqpd9qKAEav/200-cop-in-mi-studying-learned-features-in-language-models) in [200 Concrete Open Problems in Mechanistic Interpretability](https://neelnanda.io/concrete-open-problems)*

There's a surprisingly rich ecosystem of easy ways to create interactive graphics, especially for ML systems. If you're trying to do mechanistic interpretability, the ability to do web dev and to both visualize data and interact with it seems high value! 

This is a demo of how you can combine HookedTransformer and [Gradio](https://gradio.app/) to create an interactive Neuroscope - a visualization of a neuron's activations on text that will dynamically update as you edit the text. I don't particularly claim that this code is any *good*, but the goal is to illustrate what quickly hacking together a custom visualisation (while knowing fuck all about web dev, like me) can look like! (And as such, I try to explain the basic web dev concepts I use)

Note that you'll need to run the code yourself to get the interactive interface, so the cell at the bottom will be blank at first!

To emphasise - the point of this notebook is to be a rough proof of concept that just about works, *not* to be the well executed ideal of interactively studying neurons! You are highly encouraged to write your own (and ideally, to [make a pull request](https://github.com/neelnanda-io/TransformerLens/pulls) with improvements!)

## Setup

In [1]:
import os

try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    print(ipython)
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook


In [2]:
import os

if IN_COLAB:
    os.system("pip install git+https://github.com/neelnanda-io/TransformerLens.git")
    os.system("pip install gradio")

In [3]:
import gradio as gr
from transformer_lens import HookedTransformer
from transformer_lens.utils import to_numpy
from IPython.display import HTML

## Extracting Model Activations

We first write some code using HookedTransformer's cache to extract the neuron activations on a given layer and neuron, for a given text

In [4]:
model_name = "gpt2-small"
model = HookedTransformer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [5]:
model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
      (hook_resid_post): HookPoint()
    )
    (1): TransformerBlock(
      (ln1): LayerNormPre(
        (hook_sc

In [6]:
# # to be removed later 
# from transformers import GPT2Tokenizer, GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model2 = GPT2Model.from_pretrained('gpt2')
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors='pt')
# output = model2(**encoded_input)

In [7]:
from transformers.generation.utils import MODEL_FOR_SPEECH_SEQ_2_SEQ_MAPPING
MODEL_FOR_SPEECH_SEQ_2_SEQ_MAPPING.__dict__

{'_config_mapping': OrderedDict([('albert', 'AlbertConfig'),
              ('audio-spectrogram-transformer', 'ASTConfig'),
              ('bart', 'BartConfig'),
              ('beit', 'BeitConfig'),
              ('bert', 'BertConfig'),
              ('bert-generation', 'BertGenerationConfig'),
              ('big_bird', 'BigBirdConfig'),
              ('bigbird_pegasus', 'BigBirdPegasusConfig'),
              ('blenderbot', 'BlenderbotConfig'),
              ('blenderbot-small', 'BlenderbotSmallConfig'),
              ('bloom', 'BloomConfig'),
              ('camembert', 'CamembertConfig'),
              ('canine', 'CanineConfig'),
              ('chinese_clip', 'ChineseCLIPConfig'),
              ('clip', 'CLIPConfig'),
              ('clipseg', 'CLIPSegConfig'),
              ('codegen', 'CodeGenConfig'),
              ('conditional_detr', 'ConditionalDetrConfig'),
              ('convbert', 'ConvBertConfig'),
              ('convnext', 'ConvNextConfig'),
              ('ctrl', 'CTR

In [8]:
def get_neuron_acts(text, layer, neuron_index):
    # Hacky way to get out state from a single hook - we have a single element list and edit that list within the hook.
    cache = {}

    def caching_hook(act, hook):
        cache["activation"] = act[0, :, neuron_index]

    model.run_with_hooks(
        text, fwd_hooks=[(f"blocks.{layer}.mlp.hook_post", caching_hook)]
    )
    return to_numpy(cache["activation"])

We can run this function and verify that it gives vaguely sensible outputs

In [9]:
# default_layer = 9
# default_neuron_index = 652
# default_text = "The following is a list of powers of 10: 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000"
# print(model.to_str_tokens(default_text))
# print(get_neuron_acts(default_text, default_layer, default_neuron_index))

In [10]:
default_layer = 0
default_neuron_index = 3071
# default_text = "Bill Moyers criticized the corporate media for parroting the"
default_text = "Bill Moyers kritizálta a vállalati médiát mert utánozták a"
print(model.to_str_tokens(default_text))
print(get_neuron_acts(default_text, default_layer, default_neuron_index))

['<|endoftext|>', 'Bill', ' Moy', 'ers', ' k', 'rit', 'iz', 'á', 'lt', 'a', ' a', ' v', 'á', 'll', 'al', 'ati', ' m', 'é', 'di', 'á', 't', ' m', 'ert', ' ut', 'á', 'no', 'z', 't', 'á', 'k', ' a']
[ 0.02496104  0.10861742  2.7581744  -0.09472314 -0.00566751 -0.15476185
  0.1784241   0.22118981 -0.13575083  0.00575883  0.00746486  0.11344284
  0.32882148 -0.07965408 -0.17001134 -0.10030489 -0.09910711  0.3896027
 -0.1198408   0.15155126 -0.16395937 -0.06995542 -0.10853035 -0.0443168
  0.21791548 -0.1632059  -0.1537628  -0.16874783  0.24342632 -0.16870824
 -0.0390176 ]


## Visualizing Model Activations

We now write some code to visualize the neuron activations on some text - we're going to hack something together which just does some string processing to make an HTML string, with each token element colored according to the intensity neuron activation. We normalize the neuron activations so they all lie in [0, 1]. You can do much better, but this is a useful proof of concept of what "just hack stuff together" can look like!

I'll be keeping neuron 562 in layer 9 as a running example, as it seems to activate strongly on powers of 10.

Note that this visualization is very sensitive to `max_val` and `min_val`! You can tune those to whatever seems reasonable for the distribution of neuron activations you care about - I generally default to `min_val=0` and `max_val` as the max activation across the dataset.

In [11]:
# This is some CSS (tells us what style )to give each token a thin gray border, to make it easy to see token separation
style_string = """<style> 
    span.token {
        border: 1px solid rgb(123, 123, 123)
        } 
    </style>"""


def calculate_color(val, max_val, min_val):
    # Hacky code that takes in a value val in range [min_val, max_val], normalizes it to [0, 1] and returns a color which interpolates between slightly off-white and red (0 = white, 1 = red)
    # We return a string of the form "rgb(240, 240, 240)" which is a color CSS knows
    normalized_val = (val - min_val) / max_val
    return f"rgb(240, {240*(1-normalized_val)}, {240*(1-normalized_val)})"


def basic_neuron_vis(text, layer, neuron_index, max_val=None, min_val=None):
    """
    text: The text to visualize
    layer: The layer index
    neuron_index: The neuron index
    max_val: The top end of our activation range, defaults to the maximum activation
    min_val: The top end of our activation range, defaults to the minimum activation

    Returns a string of HTML that displays the text with each token colored according to its activation

    Note: It's useful to be able to input a fixed max_val and min_val, because otherwise the colors will change as you edit the text, which is annoying.
    """
    if layer is None:
        return "Please select a Layer"
    if neuron_index is None:
        return "Please select a Neuron"
    acts = get_neuron_acts(text, layer, neuron_index)
    print('acts ', acts , '\n')
    act_max = acts.max()
    act_min = acts.min()
    print('act_max :', act_max,'\n act_mix : ', act_min )
    # Defaults to the max and min of the activations
    if max_val is None:
        max_val = act_max
    if min_val is None:
        min_val = act_min
    # We want to make a list of HTML strings to concatenate into our final HTML string
    # We first add the style to make each token element have a nice border
    htmls = [style_string]
    # We then add some text to tell us what layer and neuron we're looking at 
    # - we're just dealing with strings and can use f-strings as normal
    # h4 means "small heading"
    htmls.append(f"<h4>Layer: <b>{layer}</b>. Neuron Index: <b>{neuron_index}</b></h4>")
    # We then add a line telling us the limits of our range
    htmls.append(
        f"<h4>Max Range: <b>{max_val:.4f}</b>. Min Range: <b>{min_val:.4f}</b></h4>"
    )
    # If we added a custom range, print a line telling us the range of our activations too.
    if act_max != max_val or act_min != min_val:
        htmls.append(
            f"<h4>Custom Range Set. Max Act: <b>{act_max:.4f}</b>. Min Act: <b>{act_min:.4f}</b></h4>"
        )
    # Convert the text to a list of tokens
    str_tokens = model.to_str_tokens(text)
    print('str_tokens', str_tokens)
    for tok, act in zip(str_tokens, acts):
        # A span is an HTML element that lets us style a part of a string (and remains on the same line by default)
        # We set the background color of the span to be the color we calculated from the activation
        # We set the contents of the span to be the token
        htmls.append(
            f"<span class='token' style='background-color:{calculate_color(act, max_val, min_val)}' >{tok}</span>"
        )

    return "".join(htmls)

In [12]:
# The function outputs a string of HTML
default_max_val = 4.0
default_min_val = 0.0
default_html_string = basic_neuron_vis(
    default_text,
    default_layer,
    default_neuron_index,
    max_val=default_max_val,
    min_val=default_min_val,
)

# IPython lets us display HTML
print("Displayed HTML")
display(HTML(default_html_string))

# We can also print the string directly
print("HTML String - it's just raw HTML code!")
print(default_html_string)

acts  [ 0.02496104  0.10861742  2.7581744  -0.09472314 -0.00566751 -0.15476185
  0.1784241   0.22118981 -0.13575083  0.00575883  0.00746486  0.11344284
  0.32882148 -0.07965408 -0.17001134 -0.10030489 -0.09910711  0.3896027
 -0.1198408   0.15155126 -0.16395937 -0.06995542 -0.10853035 -0.0443168
  0.21791548 -0.1632059  -0.1537628  -0.16874783  0.24342632 -0.16870824
 -0.0390176 ] 

act_max : 2.7581744 
 act_mix :  -0.17001134
str_tokens ['<|endoftext|>', 'Bill', ' Moy', 'ers', ' k', 'rit', 'iz', 'á', 'lt', 'a', ' a', ' v', 'á', 'll', 'al', 'ati', ' m', 'é', 'di', 'á', 't', ' m', 'ert', ' ut', 'á', 'no', 'z', 't', 'á', 'k', ' a']
Displayed HTML


HTML String - it's just raw HTML code!
<style> 
    span.token {
        border: 1px solid rgb(123, 123, 123)
        } 
    </style><h4>Layer: <b>0</b>. Neuron Index: <b>3071</b></h4><h4>Max Range: <b>4.0000</b>. Min Range: <b>0.0000</b></h4><h4>Custom Range Set. Max Act: <b>2.7582</b>. Min Act: <b>-0.1700</b></h4><span class='token' style='background-color:rgb(240, 238.50233785808086, 238.50233785808086)' ><|endoftext|></span><span class='token' style='background-color:rgb(240, 233.48295494914055, 233.48295494914055)' >Bill</span><span class='token' style='background-color:rgb(240, 74.50953483581543, 74.50953483581543)' > Moy</span><span class='token' style='background-color:rgb(240, 245.68338811397552, 245.68338811397552)' >ers</span><span class='token' style='background-color:rgb(240, 240.3400505427271, 240.3400505427271)' > k</span><span class='token' style='background-color:rgb(240, 249.28571105003357, 249.28571105003357)' >rit</span><span class='token' style='background-color:rg

In [13]:
print()

## Create Interactive UI

We now put all these together to create an interactive visualization in Gradio! 

The internal format is that there's a bunch of elements - Textboxes, Numbers, etc which the user can interact with and which return strings and numbers. And we can also define output elements that just display things - in this case, one which takes in an arbitrary HTML string. We call `input.change(update_function, inputs, output)` - this says "if that input element changes, run the update function on the value of each of the elements in `inputs` and set the value of `output` to the output of the function". As a bonus, this gives us live interactivity!

This is also more complex than a typical Gradio intro example - I wanted to use custom HTML to display the nice colours, which made things much messier! Normally you could just make `out` into another Textbox and pass it a string.

In [14]:
# The `with gr.Blocks() as demo:` syntax just creates a variable called demo containing all these components
with gr.Blocks() as demo:
    gr.HTML(value=f"Hacky Interactive Neuroscope for {model_name}")
    # The input elements
    with gr.Row():
        with gr.Column():
            text = gr.Textbox(label="Text", value=default_text)
            # Precision=0 makes it an int, otherwise it's a float
            # Value sets the initial default value
            layer = gr.Number(label="Layer", value=default_layer, precision=0)
            neuron_index = gr.Number(
                label="Neuron Index", value=default_neuron_index, precision=0
            )
            # If empty, these two map to None
            max_val = gr.Number(label="Max Value", value=default_max_val)
            min_val = gr.Number(label="Min Value", value=default_min_val)
            inputs = [text, layer, neuron_index, max_val, min_val]
        with gr.Column():
            # The output element
            out = gr.HTML(label="Neuron Acts", value=default_html_string)
    for inp in inputs:
        inp.change(basic_neuron_vis, inputs, out)

We can now launch our demo element, and we're done! The setting share=True even gives you a public link to the demo (though it just redirects to the backend run by this notebook, and will go away once you turn the notebook off!) Sharing makes it much slower, and can be turned off if you aren't in a colab.

**Exercise:** Explore where this neuron does and does not activate. Is it just powers of ten? Just comma separated numbers? Numbers in any particular sequence?

In [15]:
demo.launch(share=True, height=1000)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ecc1dda4-ac20-4b5f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
